# RAM-8,9 Single label

In [78]:
import numpy as np
import pandas as pd
import glob
import sys

# loading some functions we wrote before
sys.path.append("/Users/jonathanramos/Desktop/LRI/Image ROI Data Wrangling/")
from clean import *
from norm import *
from count import *

In [79]:
df = pd.read_csv('RAM-8,9_FULL_SINGLE_DB.csv')
print(df.rat_n.unique())
print(len(df.rat_n.unique()))

# removing 8-7C and 8-12C
df = df[df.rat_n != 'RAM-8-12C'].copy(deep=True)
df = df[df.rat_n != 'RAM-8-7C'].copy(deep=True)

print('\nremoving rats')
print(df.rat_n.unique())
print(len(df.rat_n.unique()))

print(np.unique(df.stain_type))

print(df.columns)
print(df.shape)
df.head()



['RAM-8-10C' 'RAM-8-12C' 'RAM-8-13C' 'RAM-8-15C' 'RAM-8-16C' 'RAM-8-1C'
 'RAM-8-3C' 'RAM-8-4C' 'RAM-8-5C' 'RAM-8-7C' 'RAM-8-8C' 'RAM-9-11B'
 'RAM-9-12B' 'RAM-9-14B' 'RAM-9-1B' 'RAM-9-2B' 'RAM-9-3B' 'RAM-9-5B'
 'RAM-9-7B' 'RAM-9-4B' 'RAM-9-8B' 'RAM-9-13B' 'RAM-9-10B']
23

removing rats
['RAM-8-10C' 'RAM-8-13C' 'RAM-8-15C' 'RAM-8-16C' 'RAM-8-1C' 'RAM-8-3C'
 'RAM-8-4C' 'RAM-8-5C' 'RAM-8-8C' 'RAM-9-11B' 'RAM-9-12B' 'RAM-9-14B'
 'RAM-9-1B' 'RAM-9-2B' 'RAM-9-3B' 'RAM-9-5B' 'RAM-9-7B' 'RAM-9-4B'
 'RAM-9-8B' 'RAM-9-13B' 'RAM-9-10B']
21
['EGFP' 'cFos' 'mKate2']
Index(['Unnamed: 0', 'rat_n', 'sex', 'dox', 'react', 'stain_type', 'filename',
       'image_name', 'CoM_x', 'CoM_y', 'mean_intensity', 'background', 'group',
       'treatment', 'mean-background'],
      dtype='object')
(27959, 15)


,Unnamed: 0,rat_n,sex,dox,react,stain_type,filename,image_name,CoM_x,CoM_y,mean_intensity,background,group,treatment,mean-background
0,0,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,345.615,79.26100000000001,141.189,56.949,RAM-8,ON_DOX_FR1,84.239990
1,1,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,57.237,77.429,140.347,56.949,RAM-8,ON_DOX_FR1,83.397995
2,2,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,13.054,117.141,255.088,56.949,RAM-8,ON_DOX_FR1,198.138990
3,3,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,368.777,231.472,92.041,56.949,RAM-8,ON_DOX_FR1,35.092000
4,4,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,107.768,29.112,194.96400000000003,56.949,RAM-8,ON_DOX_FR1,138.015000


In [80]:
cFos_min = df[df.stain_type == 'cFos']['mean-background'].min()
mKate_min = df[df.stain_type == 'mKate2']['mean-background'].min()
EGFP_min = df[df.stain_type == 'EGFP']['mean-background'].min()

print(cFos_min)
print(mKate_min)
print(EGFP_min)

def adjust_intensity(x):
    if 'cFos' in x.stain_type:
        return x['mean-background'] + abs(cFos_min)
    elif 'mKate2' in x.stain_type:
        return x['mean-background'] + abs(mKate_min)
    elif 'EGFP' in x.stain_type:
        return x['mean-background'] + abs(EGFP_min)
    else:
        print('missing val')
        return np.nan

df_adjusted = df.copy(deep=True)
df_adjusted['mean-background'] = df.apply(adjust_intensity, axis=1)

-34.80591
-126.713684
-140.9566


In [81]:
df.treatment.unique()

array(['ON_DOX_FR1', 'OFF_DOX_VR5', 'OFF_DOX_FR1', 'OFF_DOX_NR',
       'ON_DOX_VR5'], dtype=object)

# Normalizing Intensity to OFF_DOX FR1

In [82]:
for stain in df_adjusted.stain_type.unique():
    print(stain)
    df_q = df_adjusted.query(f'stain_type == "{stain}"').copy()
    norm_condition = 'OFF_DOX_FR1'
    df_norm_condition = df_q.query(f'treatment == "{norm_condition}"')
    norm_mean = df_norm_condition['mean-background'].mean()
    df_q['norm_mean-background'] = df_q['mean-background'] / norm_mean
    print(df_q.groupby('treatment')['norm_mean-background'].mean(),'\n')

    s = stain.replace('/','').replace('-','')

    prism_reorg(df_q).to_csv(f'RAM-8,9_norm_{s}_PRISM.csv')


cFos
treatment
OFF_DOX_FR1    1.000000
OFF_DOX_NR     0.896997
OFF_DOX_VR5    1.167932
ON_DOX_FR1     1.125964
ON_DOX_VR5     1.066636
Name: norm_mean-background, dtype: float64 

EGFP
treatment
OFF_DOX_FR1    1.000000
OFF_DOX_NR     1.108730
OFF_DOX_VR5    1.301446
ON_DOX_FR1     1.019303
ON_DOX_VR5     0.770104
Name: norm_mean-background, dtype: float64 

mKate2
treatment
OFF_DOX_FR1    1.000000
OFF_DOX_NR     0.919830
OFF_DOX_VR5    1.080315
ON_DOX_FR1     0.401001
ON_DOX_VR5     0.386500
Name: norm_mean-background, dtype: float64 



# Counting Mean Cell ns

In [83]:
df_img_ns = df.groupby('rat_n')['image_name'].unique().reset_index()
df_img_ns['n_imgs'] = df_img_ns.image_name.apply(len)
df_img_ns

,rat_n,image_name,n_imgs
0,RAM-8-10C,"[RAM-8-10C_3.4_PL_A, RAM-8-10C_3.9_PL_A, RAM-8...",3
1,RAM-8-13C,"[RAM-8-13C_3.1_PL_A, RAM-8-13C_3.2_PL_A, RAM-8...",5
2,RAM-8-15C,"[RAM-8-15C_3.2_PL_A, RAM-8-15C_3.2_PL_B, RAM-8...",6
3,RAM-8-16C,"[RAM-8-16C_3.1_PL_A, RAM-8-16C_3.6_PL_A, RAM-8...",4
4,RAM-8-1C,"[RAM-8-1C_2.9_PL_A, RAM-8-1C_3.0_PL_A, RAM-8-1...",6
5,RAM-8-3C,"[RAM-8-3C_3.6_PL_A, RAM-8-3C_4.0_PL_A, RAM-8-3...",3
6,RAM-8-4C,"[RAM-8-4C_3.0_PL_A, RAM-8-4C_3.2_PL_A, RAM-8-4...",3
7,RAM-8-5C,"[RAM-8-5C_3.0_PL_A, RAM-8-5C_3.3_PL_A, RAM-8-5...",5
8,RAM-8-8C,"[RAM-8-8C_3.2_PL_A, RAM-8-8C_3.4_PL_A, RAM-8-8...",3
9,RAM-9-10B,"[RAM-9_10B_A, RAM-9_10B_B, RAM-9_10B_C]",3


### All stain-type combinations

In [84]:
df_adjusted['stain'] = df_adjusted.stain_type
def mean_cell_ns_single(stain_x):
    # series of groupby's and reductions to get sum of sums per rat
    # first groupby to count cells per image; second groupby to sum counts per rat
    # merge with df to get required labels, merge with df_img_ns to get n_imgs per rat
    q = df_adjusted.query(f'stain == "{stain_x}"')\
        .groupby('image_name')['rat_n'].count()\
        .reset_index().rename(columns={'rat_n':'count'})\
        .merge(df_adjusted[['image_name', 'rat_n']], on='image_name', how='left')\
        .drop_duplicates()\
        .groupby('rat_n')['count'].sum().reset_index().rename(columns={'count':'sum'})\
        .merge(df_adjusted[['rat_n', 'react', 'dox']],\
            on='rat_n', how='left')\
        .drop_duplicates().reset_index()\
        .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

    # compute means
    q['mean_cell_n'] = q['sum'] / q['n_imgs']

    # sort by dox then react; reorder cols
    q = q.sort_values(by=['dox', 'react'])\
        [['rat_n', 'dox', 'react', 'sum', 'n_imgs', 'mean_cell_n']]\
        .reset_index().drop('index', axis=1)

    return q

df_total_cFos = mean_cell_ns_single('cFos')
df_total_mKate = mean_cell_ns_single('mKate2')

# these should be identical, but i'm performing both queries for completeness
mean_cell_ns_single('cFos').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_cFos_mean_cell_ns.csv')
mean_cell_ns_single('mKate2').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_mKate_mean_cell_ns.csv')
mean_cell_ns_single('EGFP').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_EGFP_mean_cell_ns.csv')


# splitting hi/lo cFos

In [25]:
df_cFos = df_adjusted.query('stain == "cFos"').copy()
cFos_median = df_cFos['mean-background'].median()
df_cFos['cFos_split'] = df_cFos['mean-background'].apply(lambda x: 'cFos_hi' if x > cFos_median else 'cFos_lo')


,Unnamed: 0,rat_n,sex,dox,react,stain_type,filename,image_name,CoM_x,CoM_y,mean_intensity,background,group,treatment,mean-background,stain,cFos_split
0,0,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,345.615,79.26100000000001,141.189,56.9490,RAM-8,ON_DOX_FR1,119.045900,cFos,cFos_hi
1,1,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,57.237,77.429,140.347,56.9490,RAM-8,ON_DOX_FR1,118.203905,cFos,cFos_hi
2,2,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,13.054,117.141,255.088,56.9490,RAM-8,ON_DOX_FR1,232.944900,cFos,cFos_hi
3,3,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,368.777,231.472,92.041,56.9490,RAM-8,ON_DOX_FR1,69.897910,cFos,cFos_lo
4,4,RAM-8-10C,F,ON_DOX,FR1,cFos,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,107.768,29.112,194.96400000000003,56.9490,RAM-8,ON_DOX_FR1,172.820910,cFos,cFos_hi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28967,29135,RAM-9-8B,F,OFF_DOX,FR1,cFos,RAM-9_8B_A_1.tif,RAM-9_8B_A,388.52,365.81,33.583,22.5039,RAM-9,OFF_DOX_FR1,45.885010,cFos,cFos_lo
28968,29136,RAM-9-8B,F,OFF_DOX,FR1,cFos,RAM-9_8B_A_1.tif,RAM-9_8B_A,507.71,198.03,45.9994,22.5039,RAM-9,OFF_DOX_FR1,58.301410,cFos,cFos_lo
28969,29137,RAM-9-8B,F,OFF_DOX,FR1,cFos,RAM-9_8B_A_1.tif,RAM-9_8B_A,55.59,183.08,32.3319,22.5039,RAM-9,OFF_DOX_FR1,44.633911,cFos,cFos_lo
28970,29138,RAM-9-8B,F,OFF_DOX,FR1,cFos,RAM-9_8B_A_1.tif,RAM-9_8B_A,2.65,133.29,89.8097,22.5039,RAM-9,OFF_DOX_FR1,102.111710,cFos,cFos_hi


In [29]:
def mean_cell_ns_cFosSplit(cFos_split, stain_x):
    # series of groupby's and reductions to get sum of sums per rat
    # first groupby to count cells per image; second groupby to sum counts per rat
    # merge with df to get required labels, merge with df_img_ns to get n_imgs per rat
    q = df_cFos.query(f'cFos_split == "{cFos_split}" and stain == "{stain_x}"')\
        .groupby('image_name')['rat_n'].count()\
        .reset_index().rename(columns={'rat_n':'count'})\
        .merge(df_cFos[['image_name', 'rat_n']], on='image_name', how='left')\
        .drop_duplicates()\
        .groupby('rat_n')['count'].sum().reset_index().rename(columns={'count':'sum'})\
        .merge(df_cFos[['rat_n', 'react', 'dox']],\
            on='rat_n', how='left')\
        .drop_duplicates().reset_index()\
        .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

    # compute means
    q['mean_cell_n'] = q['sum'] / q['n_imgs']

    # sort by dox then react; reorder cols
    q = q.sort_values(by=['dox', 'react'])\
        [['rat_n', 'dox', 'react', 'sum', 'n_imgs', 'mean_cell_n']]\
        .reset_index().drop('index', axis=1)

    return q

mean_cell_ns_cFosSplit('cFos_hi', 'cFos').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_cFos_hi_mean_cell_ns.csv')
mean_cell_ns_cFosSplit('cFos_lo', 'cFos').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_cFos_lo_mean_cell_ns.csv')

df_cFos_hi = mean_cell_ns_cFosSplit('cFos_hi', 'cFos').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right')
df_cFos_lo = mean_cell_ns_cFosSplit('cFos_lo', 'cFos').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right')

# RAM-8,9 Coloc labels

In [85]:
df = pd.read_csv('RAM-8,9_FULL_COLOC_DB.csv').drop('Unnamed: 0', axis=1)

# which cols have nans, how many?
print('Nan per col:')
print(df.isna().sum())
# it looks like we have 8 nans in the mean_intensity cols. If there is no
# intensity data, there's not much we can do about that; time to drop those.
df = df.dropna()

# how many duplicated rows do we have?
print('\nTotal n of duplicated rows:')
print(df.duplicated().sum())

# looks like we've got 160 duplicated rows. Let's take a look
df[df.duplicated(keep=False)]

# those duplicates look real, I'm ok with getting rid of them
df_cleaned = df.drop_duplicates().reset_index().drop('index', axis=1)
df = df_cleaned

Nan per col:
rat_n               0
sex                 0
dox                 0
react               0
stain_type          0
coloc_stain_type    0
filename            0
image_name          0
CoM_x               0
CoM_y               0
mean_intensity      0
background          0
group               0
treatment           0
dtype: int64

Total n of duplicated rows:
41


In [86]:
print(df.rat_n.unique())
print(len(df.rat_n.unique()))
print(df.treatment.unique())

# removing 8-7C and 8-12C
df = df[df.rat_n != 'RAM-8-12C'].copy(deep=True)
df = df[df.rat_n != 'RAM-8-7C'].copy(deep=True)

print('\nremoving rats')
print(df.rat_n.unique())
print(len(df.rat_n.unique()))

print(np.unique(df.coloc_stain_type))

print(df.columns)
print(df.shape)
df.head()

['RAM-8-10C' 'RAM-8-12C' 'RAM-8-13C' 'RAM-8-15C' 'RAM-8-16C' 'RAM-8-1C'
 'RAM-8-3C' 'RAM-8-4C' 'RAM-8-5C' 'RAM-8-7C' 'RAM-8-8C' 'RAM-9-11B'
 'RAM-9-12B' 'RAM-9-14B' 'RAM-9-1B' 'RAM-9-2B' 'RAM-9-3B' 'RAM-9-5B'
 'RAM-9-7B' 'RAM-9-4B' 'RAM-9-13B' 'RAM-9-8B' 'RAM-9-10B']
23
['ON_DOX_FR1' 'OFF_DOX_VR5' 'OFF_DOX_FR1' 'OFF_DOX_NR' 'ON_DOX_VR5']

removing rats
['RAM-8-10C' 'RAM-8-13C' 'RAM-8-15C' 'RAM-8-16C' 'RAM-8-1C' 'RAM-8-3C'
 'RAM-8-4C' 'RAM-8-5C' 'RAM-8-8C' 'RAM-9-11B' 'RAM-9-12B' 'RAM-9-14B'
 'RAM-9-1B' 'RAM-9-2B' 'RAM-9-3B' 'RAM-9-5B' 'RAM-9-7B' 'RAM-9-4B'
 'RAM-9-13B' 'RAM-9-8B' 'RAM-9-10B']
21
['EGFP_coloc_w_cFos' 'EGFP_coloc_w_mKate2' 'cFos_coloc_w_EGFP'
 'cFos_coloc_w_mKate2' 'mKate2_coloc_w_EGFP' 'mKate2_coloc_w_cFos'
 'triple_EGFP' 'triple_cFos' 'triple_mKate2']
Index(['rat_n', 'sex', 'dox', 'react', 'stain_type', 'coloc_stain_type',
       'filename', 'image_name', 'CoM_x', 'CoM_y', 'mean_intensity',
       'background', 'group', 'treatment'],
      dtype='object')
(10502, 14)


,rat_n,sex,dox,react,stain_type,coloc_stain_type,filename,image_name,CoM_x,CoM_y,mean_intensity,background,group,treatment
0,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,155.840,26.785,141.465,56.949,RAM-8,ON_DOX_FR1
1,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,206.780,79.099,168.818,56.949,RAM-8,ON_DOX_FR1
2,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,368.777,231.472,92.041,56.949,RAM-8,ON_DOX_FR1
3,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,408.535,6.850,107.794,56.949,RAM-8,ON_DOX_FR1
4,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,13.054,117.141,255.088,56.949,RAM-8,ON_DOX_FR1


In [87]:
def get_stain_type(x):
    if 'triple' in x:
        return x.split('_')[-1]
    elif not 'triple' in x:
        return x.split('_')[0]

df['stain_type'] = df['coloc_stain_type'].apply(get_stain_type)
print(df.stain_type.unique())

['cFos' 'EGFP' 'mKate2']


## Adjusting raw mean-background intensity

In [88]:
print(cFos_min)
print(mKate_min)
print(EGFP_min)

df['mean-background'] = df['mean_intensity'] - df['background']

def adjust_intensity(x):
    if 'cFos' in x.stain_type:
        return x['mean-background'] + abs(cFos_min)
    elif 'mKate2' in x.stain_type:
        return x['mean-background'] + abs(mKate_min)
    elif 'EGFP' in x.stain_type:
        return x['mean-background'] + abs(EGFP_min)
    else:
        print('missing val')
        return np.nan

df_adjusted = df.copy(deep=True)
df_adjusted['mean-background'] = df.apply(adjust_intensity, axis=1)

-34.80591
-126.713684
-140.9566


# Normalzing intensity to OFF_DOX_FR1

In [89]:
for stain in df_adjusted.coloc_stain_type.unique():
    print(stain)
    df_q = df_adjusted.query(f'coloc_stain_type == "{stain}"').copy()
    norm_condition = 'OFF_DOX_FR1'
    df_norm_condition = df_q.query(f'treatment == "{norm_condition}"')
    norm_mean = df_norm_condition['mean-background'].mean()
    df_q['norm_mean-background'] = df_q['mean-background'] / norm_mean
    print(df_q.groupby('treatment')['norm_mean-background'].mean(),'\n')

    s = stain.replace('/','').replace('-','')

    prism_reorg(df_q).to_csv(f'RAM-8,9_norm_{s}_PRISM.csv')

cFos_coloc_w_EGFP
treatment
OFF_DOX_FR1    1.000000
OFF_DOX_NR     0.728401
OFF_DOX_VR5    1.154708
ON_DOX_FR1     1.160507
ON_DOX_VR5     0.829338
Name: norm_mean-background, dtype: float64 

EGFP_coloc_w_cFos
treatment
OFF_DOX_FR1    1.000000
OFF_DOX_NR     1.511820
OFF_DOX_VR5    1.226041
ON_DOX_FR1     0.962850
ON_DOX_VR5     0.801094
Name: norm_mean-background, dtype: float64 

cFos_coloc_w_mKate2
treatment
OFF_DOX_FR1    1.000000
OFF_DOX_NR     0.712752
OFF_DOX_VR5    1.063702
ON_DOX_FR1     1.250554
Name: norm_mean-background, dtype: float64 

mKate2_coloc_w_cFos
treatment
OFF_DOX_FR1    1.000000
OFF_DOX_NR     1.252045
OFF_DOX_VR5    1.095368
ON_DOX_FR1     0.353164
Name: norm_mean-background, dtype: float64 

mKate2_coloc_w_EGFP
treatment
OFF_DOX_FR1    1.000000
OFF_DOX_NR     0.886241
OFF_DOX_VR5    1.022507
ON_DOX_FR1     0.356797
ON_DOX_VR5     0.352758
Name: norm_mean-background, dtype: float64 

triple_cFos
treatment
OFF_DOX_FR1    1.000000
OFF_DOX_NR     0.636353
OFF_DOX

# Counting Mean Cell ns

In [90]:
df_adjusted['stain'] = df_adjusted.coloc_stain_type
def mean_cell_ns(stain_x):
    # series of groupby's and reductions to get sum of sums per rat
    # first groupby to count cells per image; second groupby to sum counts per rat
    # merge with df to get required labels, merge with df_img_ns to get n_imgs per rat
    q = df_adjusted.query(f'stain == "{stain_x}"')\
        .groupby('image_name')['rat_n'].count()\
        .reset_index().rename(columns={'rat_n':'count'})\
        .merge(df_adjusted[['image_name', 'rat_n']], on='image_name', how='left')\
        .drop_duplicates()\
        .groupby('rat_n')['count'].sum().reset_index().rename(columns={'count':'sum'})\
        .merge(df_adjusted[['rat_n', 'react', 'dox']],\
            on='rat_n', how='left')\
        .drop_duplicates().reset_index()\
        .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

    # compute means
    q['mean_cell_n'] = q['sum'] / q['n_imgs']

    # sort by dox then react; reorder cols
    q = q.sort_values(by=['dox', 'react'])\
        [['rat_n', 'dox', 'react', 'sum', 'n_imgs', 'mean_cell_n']]\
        .reset_index().drop('index', axis=1)

    return q

# these should be identical, but i'm performing both queries for completeness
for stain in df_adjusted.stain.unique():
    mean_cell_ns(f'{stain}').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv(f'RAM-8,9_{stain}_mean_cell_ns.csv')

# splitting hi/lo cFos

In [91]:
df_cFos = df_adjusted.query('stain_type == "cFos"').copy()
df_cFos['cFos_split'] = df_cFos['mean-background'].apply(lambda x: 'cFos_hi' if x > cFos_median else 'cFos_lo')
df_cFos


,rat_n,sex,dox,react,stain_type,coloc_stain_type,filename,image_name,CoM_x,CoM_y,mean_intensity,background,group,treatment,mean-background,stain,cFos_split
0,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,155.840,26.785,141.4650,56.9490,RAM-8,ON_DOX_FR1,119.32191,cFos_coloc_w_EGFP,cFos_hi
1,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,206.780,79.099,168.8180,56.9490,RAM-8,ON_DOX_FR1,146.67491,cFos_coloc_w_EGFP,cFos_hi
2,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,368.777,231.472,92.0410,56.9490,RAM-8,ON_DOX_FR1,69.89791,cFos_coloc_w_EGFP,cFos_lo
3,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,408.535,6.850,107.7940,56.9490,RAM-8,ON_DOX_FR1,85.65091,cFos_coloc_w_EGFP,cFos_lo
4,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,13.054,117.141,255.0880,56.9490,RAM-8,ON_DOX_FR1,232.94491,cFos_coloc_w_EGFP,cFos_hi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10567,RAM-9-4B,M,OFF_DOX,NR,cFos,triple_cFos,RAM-9_4B_E_1.tif,RAM-9_4B_E,153.990,102.840,293.6942,177.3912,RAM-9,OFF_DOX_NR,151.10891,triple_cFos,cFos_hi
10568,RAM-9-4B,M,OFF_DOX,NR,cFos,triple_cFos,RAM-9_4B_E_1.tif,RAM-9_4B_E,288.020,135.360,393.3342,177.3912,RAM-9,OFF_DOX_NR,250.74891,triple_cFos,cFos_hi
10569,RAM-9-4B,M,OFF_DOX,NR,cFos,triple_cFos,RAM-9_4B_E_1.tif,RAM-9_4B_E,407.850,58.610,399.4489,177.3912,RAM-9,OFF_DOX_NR,256.86361,triple_cFos,cFos_hi
10592,RAM-9-8B,F,OFF_DOX,FR1,cFos,triple_cFos,RAM-9_8B_D_1.tif,RAM-9_8B_D,405.500,223.610,76.3239,17.0164,RAM-9,OFF_DOX_FR1,94.11341,triple_cFos,cFos_lo


In [92]:
def mean_cell_ns_cFosSplit(cFos_split, stain_x):
    # series of groupby's and reductions to get sum of sums per rat
    # first groupby to count cells per image; second groupby to sum counts per rat
    # merge with df to get required labels, merge with df_img_ns to get n_imgs per rat
    q = df_cFos.query(f'cFos_split == "{cFos_split}" and stain == "{stain_x}"')\
        .groupby('image_name')['rat_n'].count()\
        .reset_index().rename(columns={'rat_n':'count'})\
        .merge(df_cFos[['image_name', 'rat_n']], on='image_name', how='left')\
        .drop_duplicates()\
        .groupby('rat_n')['count'].sum().reset_index().rename(columns={'count':'sum'})\
        .merge(df_cFos[['rat_n', 'react', 'dox']],\
            on='rat_n', how='left')\
        .drop_duplicates().reset_index()\
        .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

    # compute means
    q['mean_cell_n'] = q['sum'] / q['n_imgs']

    # sort by dox then react; reorder cols
    q = q.sort_values(by=['dox', 'react'])\
        [['rat_n', 'dox', 'react', 'sum', 'n_imgs', 'mean_cell_n']]\
        .reset_index().drop('index', axis=1)

    return q

mean_cell_ns_cFosSplit('cFos_hi', 'cFos_coloc_w_mKate2').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_cFos_hi_coloc_w_mKate2_mean_cell_ns.csv')
mean_cell_ns_cFosSplit('cFos_lo', 'cFos_coloc_w_mKate2').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_cFos_lo_coloc_w_mKate2_mean_cell_ns.csv')

In [97]:
df_cFos_hi_w_mKate = mean_cell_ns_cFosSplit('cFos_hi', 'cFos_coloc_w_mKate2')
df_cFos_lo_w_mKate = mean_cell_ns_cFosSplit('cFos_lo', 'cFos_coloc_w_mKate2')

df_cFos_hi['total_cFos_hi'] = df_cFos_hi['sum']
df_cFos_lo['total_cFos_lo'] = df_cFos_lo['sum']

df_cFos_hi_w_mKate['total_cFos_hi_coloc_w_mKate'] = df_cFos_hi_w_mKate['sum']
df_cFos_lo_w_mKate['total_cFos_lo_coloc_w_mKate'] = df_cFos_lo_w_mKate['sum']

df_percent_cFos_hi_w_mKate = df_cFos_hi_w_mKate[['rat_n', 'dox', 'react', 'total_cFos_hi_coloc_w_mKate']].merge(df_cFos_hi[['rat_n', 'dox', 'react', 'total_cFos_hi']], on=['rat_n', 'dox', 'react'], how='left')
df_percent_cFos_hi_w_mKate['percent'] = df_percent_cFos_hi_w_mKate.total_cFos_hi_coloc_w_mKate / df_percent_cFos_hi_w_mKate.total_cFos_hi * 100
df_percent_cFos_hi_w_mKate.merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_TOTAL_cFos_hi_percent_mKate_overlap.csv')

df_percent_cFos_lo_w_mKate = df_cFos_lo_w_mKate[['rat_n', 'dox', 'react', 'total_cFos_lo_coloc_w_mKate']].merge(df_cFos_lo[['rat_n', 'dox', 'react', 'total_cFos_lo']], on=['rat_n', 'dox', 'react'], how='left')
df_percent_cFos_lo_w_mKate['percent'] = df_percent_cFos_lo_w_mKate.total_cFos_lo_coloc_w_mKate / df_percent_cFos_lo_w_mKate.total_cFos_lo * 100
df_percent_cFos_lo_w_mKate.merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_TOTAL_cFos_lo_percent_mKate_overlap.csv')

In [98]:
df_total_mKate['total_mKate'] = df_total_mKate['sum']

df_cFos_hi_w_mKate['total_mKate_coloc_w_cFos_hi'] = df_cFos_hi_w_mKate['sum']
df_cFos_lo_w_mKate['total_mKate_coloc_w_cFos_lo'] = df_cFos_lo_w_mKate['sum']
df_mKate_w_cFos_hi = df_cFos_hi_w_mKate.drop('total_cFos_hi_coloc_w_mKate',axis=1).copy(deep=True)
df_mKate_w_cFos_lo = df_cFos_lo_w_mKate.drop('total_cFos_lo_coloc_w_mKate',axis=1).copy(deep=True)

df_percent_mKate_w_cFos_hi = df_mKate_w_cFos_hi[['rat_n', 'dox', 'react', 'total_mKate_coloc_w_cFos_hi']].merge(df_total_mKate[['rat_n', 'dox', 'react', 'total_mKate']], on=['rat_n', 'dox', 'react'], how='left')
df_percent_mKate_w_cFos_hi['percent'] = df_percent_mKate_w_cFos_hi.total_mKate_coloc_w_cFos_hi / df_percent_mKate_w_cFos_hi.total_mKate * 100
df_percent_mKate_w_cFos_hi.merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_TOTAL_mKate_percent_cFos_hi_overlap.csv')

df_percent_mKate_w_cFos_lo = df_mKate_w_cFos_lo[['rat_n', 'dox', 'react', 'total_mKate_coloc_w_cFos_lo']].merge(df_total_mKate[['rat_n', 'dox', 'react', 'total_mKate']], on=['rat_n', 'dox', 'react'], how='left')
df_percent_mKate_w_cFos_lo['percent'] = df_percent_mKate_w_cFos_lo.total_mKate_coloc_w_cFos_lo / df_percent_mKate_w_cFos_lo.total_mKate * 100
df_percent_mKate_w_cFos_lo.merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_TOTAL_mKate_percent_cFos_lo_overlap.csv')

In [99]:
df_total_mKate

,rat_n,dox,react,sum,n_imgs,mean_cell_n,total_mKate
0,RAM-8-15C,OFF_DOX,FR1,120,6,20.000000,120
1,RAM-8-4C,OFF_DOX,FR1,135,3,45.000000,135
2,RAM-8-8C,OFF_DOX,FR1,91,3,30.333333,91
3,RAM-9-11B,OFF_DOX,FR1,340,3,113.333333,340
4,RAM-9-3B,OFF_DOX,FR1,226,4,56.500000,226
5,RAM-9-8B,OFF_DOX,FR1,247,4,61.750000,247
6,RAM-8-1C,OFF_DOX,NR,48,6,8.000000,48
7,RAM-9-10B,OFF_DOX,NR,113,3,37.666667,113
8,RAM-9-13B,OFF_DOX,NR,580,6,96.666667,580
9,RAM-9-4B,OFF_DOX,NR,677,6,112.833333,677


In [113]:
df_cFos_on_mKate = mean_cell_ns(f'cFos_coloc_w_mKate2').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right')
df_mKate_on_cFos = mean_cell_ns(f'mKate2_coloc_w_cFos').merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right')

df_cFos_on_mKate['total_cFos_on_mKate'] = df_cFos_on_mKate['sum']
df_mKate_on_cFos['total_mKate_on_cFos'] = df_mKate_on_cFos['sum']

df_total_cFos['total_cFos'] = df_total_cFos['sum']
df_percent_cFos_on_mKate = df_cFos_on_mKate[['rat_n', 'dox', 'react', 'total_cFos_on_mKate']].merge(df_total_cFos[['rat_n', 'dox', 'react', 'total_cFos']], on=['rat_n', 'dox', 'react'], how='left')
df_percent_cFos_on_mKate['percent'] = df_percent_cFos_on_mKate.total_cFos_on_mKate / df_percent_cFos_on_mKate.total_cFos * 100
df_percent_cFos_on_mKate.merge(df_total_mKate[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_TOTAL_cFos_percent_mKate_overlap.csv')

df_percent_mKate_on_cFos = df_mKate_on_cFos[['rat_n', 'dox', 'react', 'total_mKate_on_cFos']].merge(df_total_mKate[['rat_n', 'dox', 'react', 'total_mKate']], on=['rat_n', 'dox', 'react'], how='left')
df_percent_mKate_on_cFos['percent'] = df_percent_mKate_on_cFos.total_mKate_on_cFos / df_percent_mKate_on_cFos.total_mKate * 100
df_percent_mKate_on_cFos.merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-8,9_TOTAL_mKate_percent_cFos_overlap.csv')

In [114]:
df_cFos

,rat_n,sex,dox,react,stain_type,coloc_stain_type,filename,image_name,CoM_x,CoM_y,mean_intensity,background,group,treatment,mean-background,stain,cFos_split
0,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,155.840,26.785,141.4650,56.9490,RAM-8,ON_DOX_FR1,119.32191,cFos_coloc_w_EGFP,cFos_hi
1,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,206.780,79.099,168.8180,56.9490,RAM-8,ON_DOX_FR1,146.67491,cFos_coloc_w_EGFP,cFos_hi
2,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,368.777,231.472,92.0410,56.9490,RAM-8,ON_DOX_FR1,69.89791,cFos_coloc_w_EGFP,cFos_lo
3,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,408.535,6.850,107.7940,56.9490,RAM-8,ON_DOX_FR1,85.65091,cFos_coloc_w_EGFP,cFos_lo
4,RAM-8-10C,F,ON_DOX,FR1,cFos,cFos_coloc_w_EGFP,RAM-8-10C_3.4_PL_A_2.tif,RAM-8-10C_3.4_PL_A,13.054,117.141,255.0880,56.9490,RAM-8,ON_DOX_FR1,232.94491,cFos_coloc_w_EGFP,cFos_hi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10567,RAM-9-4B,M,OFF_DOX,NR,cFos,triple_cFos,RAM-9_4B_E_1.tif,RAM-9_4B_E,153.990,102.840,293.6942,177.3912,RAM-9,OFF_DOX_NR,151.10891,triple_cFos,cFos_hi
10568,RAM-9-4B,M,OFF_DOX,NR,cFos,triple_cFos,RAM-9_4B_E_1.tif,RAM-9_4B_E,288.020,135.360,393.3342,177.3912,RAM-9,OFF_DOX_NR,250.74891,triple_cFos,cFos_hi
10569,RAM-9-4B,M,OFF_DOX,NR,cFos,triple_cFos,RAM-9_4B_E_1.tif,RAM-9_4B_E,407.850,58.610,399.4489,177.3912,RAM-9,OFF_DOX_NR,256.86361,triple_cFos,cFos_hi
10592,RAM-9-8B,F,OFF_DOX,FR1,cFos,triple_cFos,RAM-9_8B_D_1.tif,RAM-9_8B_D,405.500,223.610,76.3239,17.0164,RAM-9,OFF_DOX_FR1,94.11341,triple_cFos,cFos_lo
